In [2]:
# Data & geospatial packages
import os
import glob
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
import numpy as np

# Progress bar
from tqdm import tqdm

# For datetime parsing
from datetime import datetime


In [4]:
# File paths
shapefile_path = r"C:\Users\jbros\OneDrive\Documents\SanDiego_RemoteSensing\Agricultural_Commodity_2020_shapefile\Agricultural_Commodity_2020.shp"
csv_path = r"C:\Users\jbros\OneDrive\Documents\SanDiego_RemoteSensing\AFoster_data_merged.csv"
raster_dir = r"C:\Users\jbros\OneDrive\Documents\SanDiego_RemoteSensing\San_LST_20182025_cloudmasked_PST"
output_dir = r"C:\Users\jbros\OneDrive\Documents\SanDiego_RemoteSensing\raster_stats"


In [6]:
# Load the shapefile as a GeoDataFrame
gdf = gpd.read_file(shapefile_path)
print(f" Shapefile loaded with {len(gdf)} records.")
print("Columns:", gdf.columns.tolist())

# Load the AFoster merged data
df = pd.read_csv(csv_path)
print(f" CSV loaded with {len(df)} records.")
print("Columns:", df.columns.tolist())


 Shapefile loaded with 2394 records.
Columns: ['objectid', 'site_id', 'permit_num', 'permittee', 'site_name', 'loc_narr', 'quantity', 'units', 'commodity', 'calc_acres', 'mtrs', 'globalid', 'shape_Leng', 'shape_Area', 'geometry']
 CSV loaded with 10254 records.
Columns: ['site_id', 'mean_height', 'max_height', 'z_bin_hgtcateg', 'tree_ct', 'total_trees', 'calc_acres', 'calc_hectares', 'dens_ha_total', 'dens_ha_binned', 'binned_percent']


In [8]:
# Merge GeoDataFrame and DataFrame on 'site_id'
if 'site_id' in gdf.columns and 'site_id' in df.columns:
    gdf = gdf.merge(df, on='site_id')
    print(f" Merge successful. Resulting GeoDataFrame has {gdf.shape[0]} rows and {gdf.shape[1]} columns.")
else:
    raise ValueError(" 'site_id' column missing in either the shapefile or the CSV.")


 Merge successful. Resulting GeoDataFrame has 16774 rows and 25 columns.


In [10]:
# Get list of all .tif raster files in the ECOSTRESS directory (recursively)
raster_files = glob.glob(os.path.join(raster_dir, "**", "*.tif"), recursive=True)

# Report how many files were found
print(f" Found {len(raster_files)} raster files in: {raster_dir}")

# Show the first few file names for verification
raster_files[:5]


 Found 295 raster files in: C:\Users\jbros\OneDrive\Documents\SanDiego_RemoteSensing\San_LST_20182025_cloudmasked_PST


['C:\\Users\\jbros\\OneDrive\\Documents\\SanDiego_RemoteSensing\\San_LST_20182025_cloudmasked_PST\\2018.07.13.16.43.42_ST.tif',
 'C:\\Users\\jbros\\OneDrive\\Documents\\SanDiego_RemoteSensing\\San_LST_20182025_cloudmasked_PST\\2018.07.14.23.59.20_ST.tif',
 'C:\\Users\\jbros\\OneDrive\\Documents\\SanDiego_RemoteSensing\\San_LST_20182025_cloudmasked_PST\\2018.07.24.20.07.39_ST.tif',
 'C:\\Users\\jbros\\OneDrive\\Documents\\SanDiego_RemoteSensing\\San_LST_20182025_cloudmasked_PST\\2018.07.30.10.01.45_ST.tif',
 'C:\\Users\\jbros\\OneDrive\\Documents\\SanDiego_RemoteSensing\\San_LST_20182025_cloudmasked_PST\\2018.08.06.07.10.46_ST.tif']

In [12]:
def extract_datetime_from_filename(filepath):
    """
    Extracts a datetime object from a raster filename like:
    '2018.07.13.16.43.42_ST.tif'
    """
    try:
        basename = os.path.basename(filepath)
        date_str = basename.split("_")[0]  # '2018.07.13.16.43.42'
        dt = datetime.strptime(date_str, "%Y.%m.%d.%H.%M.%S")
        return dt
    except Exception as e:
        print(f" Could not parse datetime from {filepath}: {e}")
        return None


In [14]:
def compute_zonal_stats(raster_path, gdf, output_dir=None):
    """
    Compute zonal statistics (mean and std) for each polygon in gdf
    from the given raster. Includes datetime extracted from filename.
    """
    stats = []
    dt = extract_datetime_from_filename(raster_path)

    try:
        with rasterio.open(raster_path) as src:
            for i, geom in enumerate(gdf.geometry):
                try:
                    out_image, out_transform = mask(src, [geom], crop=True)
                    data = out_image[0]
                    data = data[data != src.nodata]  # Remove no-data

                    if data.size > 0:
                        stats.append({
                            "site_id": gdf.iloc[i]["site_id"],
                            "mean_LST": np.mean(data),
                            "std_LST": np.std(data),
                            "datetime": dt
                        })
                except Exception as geom_err:
                    print(f" Skipping geometry {i}: {geom_err}")
    except Exception as raster_err:
        print(f" Error reading raster {raster_path}: {raster_err}")

    df_stats = pd.DataFrame(stats)

    # Optionally save individual file output
    if output_dir and not df_stats.empty:
        out_name = os.path.basename(raster_path).replace(".tif", "_zonal.csv")
        out_path = os.path.join(output_dir, out_name)
        df_stats.to_csv(out_path, index=False)

    return df_stats


In [16]:
# Check CRS of first raster
with rasterio.open(raster_files[0]) as src:
    raster_crs = src.crs
    print(f" Raster CRS: {raster_crs}")

# Check CRS of GeoDataFrame
print(f"📐 Shapefile CRS: {gdf.crs}")

# Reproject shapefile if needed
if gdf.crs != raster_crs:
    gdf = gdf.to_crs(raster_crs)
    print("Reprojected GeoDataFrame to match raster CRS.")
else:
    print(" CRS already aligned. No reprojection needed.")


 Raster CRS: EPSG:4326
📐 Shapefile CRS: EPSG:2230
Reprojected GeoDataFrame to match raster CRS.


In [18]:
# Create a list to store all per-raster DataFrames
all_stats = []

# Process all rasters with progress bar
for raster_path in tqdm(raster_files, desc="Computing zonal stats"):
    try:
        df = compute_zonal_stats(raster_path, gdf, output_dir=output_dir)
        if not df.empty:
            all_stats.append(df)
    except Exception as e:
        print(f"⚠️ Failed on {os.path.basename(raster_path)} — {e}")

# Combine all into one master DataFrame
if all_stats:
    master_df = pd.concat(all_stats, ignore_index=True)
    output_csv = os.path.join(output_dir, "ECOSTRESS_zonal_all_parcels.csv")
    master_df.to_csv(output_csv, index=False)
    print(f"\n✅ Saved master zonal stats to:\n{output_csv}")
    display(master_df.head())
else:
    print("❗ No zonal statistics were generated.")


Computing zonal stats:   0%|          | 0/295 [00:00<?, ?it/s]

 Skipping geometry 6920: Input shapes do not overlap raster.
 Skipping geometry 6921: Input shapes do not overlap raster.
 Skipping geometry 6922: Input shapes do not overlap raster.
 Skipping geometry 9528: Input shapes do not overlap raster.
 Skipping geometry 14921: Input shapes do not overlap raster.
 Skipping geometry 14922: Input shapes do not overlap raster.
 Skipping geometry 14923: Input shapes do not overlap raster.


Computing zonal stats:   0%|          | 1/295 [01:03<5:11:25, 63.56s/it]

 Skipping geometry 6920: Input shapes do not overlap raster.
 Skipping geometry 6921: Input shapes do not overlap raster.
 Skipping geometry 6922: Input shapes do not overlap raster.
 Skipping geometry 9528: Input shapes do not overlap raster.
 Skipping geometry 14921: Input shapes do not overlap raster.
 Skipping geometry 14922: Input shapes do not overlap raster.
 Skipping geometry 14923: Input shapes do not overlap raster.


Computing zonal stats:   1%|          | 2/295 [02:06<5:08:05, 63.09s/it]

...

Computing zonal stats: 100%|█████████▉| 294/295 [6:36:07<01:19, 79.99s/it]

 Skipping geometry 6920: Input shapes do not overlap raster.
 Skipping geometry 6921: Input shapes do not overlap raster.
 Skipping geometry 6922: Input shapes do not overlap raster.
 Skipping geometry 9528: Input shapes do not overlap raster.
 Skipping geometry 14921: Input shapes do not overlap raster.
 Skipping geometry 14922: Input shapes do not overlap raster.
 Skipping geometry 14923: Input shapes do not overlap raster.


Computing zonal stats: 100%|██████████| 295/295 [6:37:27<00:00, 80.84s/it]



✅ Saved master zonal stats to:
C:\Users\jbros\OneDrive\Documents\SanDiego_RemoteSensing\raster_stats\ECOSTRESS_zonal_all_parcels.csv


,site_id,mean_LST,std_LST,datetime
0,AWC001,21.258656,0.471390,2018-07-13 16:43:42
1,AWC001,21.258656,0.471390,2018-07-13 16:43:42
2,AWC001,21.258656,0.471390,2018-07-13 16:43:42
3,AWW023,21.542725,0.484394,2018-07-13 16:43:42
4,AWW023,21.542725,0.484394,2018-07-13 16:43:42


In [13]:

# Cell 2: Set folder path
folder_path = r"C:\Users\jbros\OneDrive\Documents\SanDiego_RemoteSensing\raster_stats"
csv_files = glob(os.path.join(folder_path, "*.csv"))

# Cell 3: Read and collect all data
all_data = []

for file in csv_files:
    filename = os.path.basename(file)
    datetime_str = filename.split("_")[0]  # e.g., 2018.07.13.16.43.42
    try:
        dt_parts = datetime_str.split(".")
        full_datetime = f"{dt_parts[0]}-{dt_parts[1]}-{dt_parts[2]} {dt_parts[3]}:{dt_parts[4]}:{dt_parts[5]}"
    except Exception as e:
        print(f"Filename parse error: {filename}")
        continue

    try:
        df = pd.read_csv(file)
        df['datetime'] = full_datetime
        all_data.append(df[['site_id', 'datetime', 'mean_LST', 'std_LST']])
    except Exception as e:
        print(f"Read error: {file}: {e}")

# Cell 4: Combine all files
combined = pd.concat(all_data, ignore_index=True)

# Cell 5: Pivot to wide format (use average in case of duplicates)
mean_wide = combined.pivot_table(index='site_id', columns='datetime', values='mean_LST', aggfunc='mean')
std_wide  = combined.pivot_table(index='site_id', columns='datetime', values='std_LST', aggfunc='mean')

# Optional: Sort columns chronologically
mean_wide = mean_wide[sorted(mean_wide.columns)]
std_wide = std_wide[sorted(std_wide.columns)]

# Cell 6: Save as two separate CSVs
mean_csv_path = os.path.join(folder_path, "LST_mean_wide.csv")
std_csv_path = os.path.join(folder_path, "LST_std_wide.csv")

mean_wide.to_csv(mean_csv_path)
std_wide.to_csv(std_csv_path)

print(f"Saved mean LST CSV to: {mean_csv_path}")
print(f"Saved std LST CSV to: {std_csv_path}")


Filename parse error: combined_zonal_summary.csv
Filename parse error: pivoted_LST_stats_wide.csv
Saved mean LST CSV to: C:\Users\jbros\OneDrive\Documents\SanDiego_RemoteSensing\raster_stats\LST_mean_wide.csv
Saved std LST CSV to: C:\Users\jbros\OneDrive\Documents\SanDiego_RemoteSensing\raster_stats\LST_std_wide.csv
